# Celery: First steps

RabbitMQ Management: http://localhost:15672/#/
And open redis client manager

## Step #1: Install celery

In [ ]:
!pip install celery

# Step #2: Celery app

Code in file [tasks.py](tasks.py)

# Step #3: Run Celery worker server

In [ ]:
!celery -A tasks worker --loglevel=info

 
 -------------- celery@cocombra v4.3.0 (rhubarb)
---- **** ----- 
--- * ***  * -- Linux-4.18.0-25-generic-x86_64-with-Ubuntu-18.04-bionic 2019-07-15 17:18:04
-- * - **** --- 
- ** ---------- [config]
- ** ---------- .> app:         tasks:0x7f02da340eb8
- ** ---------- .> transport:   amqp://guest:**@localhost:5672//
- ** ---------- .> results:     redis://localhost/
- *** --- * --- .> concurrency: 12 (prefork)
-- ******* ---- .> task events: OFF (enable -E to monitor tasks in this worker)
--- ***** ----- 
 -------------- [queues]
                .> celery           exchange=celery(direct) key=celery
                

[tasks]
  . tasks.add
  . tasks.mul

[2019-07-15 17:18:05,078: INFO/MainProcess] Connected to amqp://guest:**@127.0.0.1:5672//
[2019-07-15 17:18:05,082: INFO/MainProcess] mingle: searching for neighbors
[2019-07-15 17:18:06,129: INFO/MainProcess] mingle: all alone
[2019-07-15 17:18:06,161: INFO/MainProcess] celery@cocombra ready.
[2019-07-15 17:18:06,162: INFO/MainProces

# Step #4: Calling the task

Open rabbitmq manager to see the transactions in the queues: http://localhost:15672/#/queues/%2F/celery

In [1]:
from tasks import app, add, mul

### apply_async() 
Apply tasks asynchronously by sending a message.

In [5]:
add.apply_async((4, 4), countdown=10)

<AsyncResult: 23933bc1-a5ec-4910-b5e7-620292890cfe>

In [4]:
add.apply_async((4, 4)).get()

8

### delay()
Star argument version of `apply_async()` that does not support the extra options.
The `delay()` method is convenient as it looks like calling a regular function

In [ ]:
add.delay(4, 4)

In [ ]:
add.delay(4, 4).get()

### send_task()
Send task by name.
Supports the same arguments as `apply_async()`

In [ ]:
app.send_task('tasks.add', args=[2,2])

In [ ]:
app.send_task('tasks.add', args=[2,2]).get()

# Step #5: Keeping Results

In [ ]:
result = add.delay(4, 4)
result.ready()

In [ ]:
result.get()

In [ ]:
print(result)

In [ ]:
!redis-cli get 'celery-task-meta-d58495cd-4e79-452b-9ad5-4127b4501c46'

# Canvas: Designing workflows

### Chains

In [15]:
res = add.apply_async((2, 2), link=mul.s(16))
res

<AsyncResult: fe202c5d-e41b-49ef-adfe-94c00752cb07>

In [17]:
res.children[0].get()

64

In [18]:
s = add.s(2, 2)
s.link(mul.s(4))

res = s.delay()
res.get()

<AsyncResult: 3cf03c02-ee7c-4869-820a-5cc0a04d5af4>

In [21]:
from celery import chain

res = chain(add.s(4, 4), mul.s(8), mul.s(10))()
res.get()

640

In [22]:
res.parent.get()

64

In [23]:
res.parent.parent.get()

8

Chains can also be made using the | (pipe) operator:

In [ ]:
(add.s(2, 2) | mul.s(8) | mul.s(10)).apply_async().get()

### Groups

In [24]:
from celery import group
from tasks import add

group(add.s(2, 2), add.s(4, 4))().get()

[4, 8]

In [25]:
group(add.s(i, i) for i in range(10000))()

<GroupResult: 0df4f0f4-c99f-4b07-99db-ab0e252eaf2f [9fff8519-b64b-428b-bd61-27568b8463ef, 5efe1dac-3ee3-4d02-9566-b147db2caf78, 01f74597-f3ae-484c-93f4-7f048068a78f, cd1717df-aacb-416a-a655-652d89021349, d7d64c01-f06a-4caa-a1ae-fe9cac81331c, 7888e88d-cd9d-47e4-9a38-0512eb8caf10, 188cc55c-a651-4d44-993e-96f86e7620fd, 4313c287-58ac-4caa-b6ce-b24fa9666a82, 50f8199c-003e-47dd-b1d2-1c95e37b4ca4, 1dfb39ef-ac3b-49b3-bd59-36da71ecac6f, 0be9abb4-1307-4781-a322-4216dcb44d23, 35cd4608-26cd-4f6b-b189-522c6cf58508, 2836e553-1aac-4444-9ede-33173857b11a, bdad43b7-f51d-4ca3-9603-a19e2661771d, c9419015-ad62-4f3a-8ed4-23745fc0f84b, 328365ba-76fa-4200-a69c-b0f9d03e73d6, d2c91f30-993e-4443-a685-af79ae4dd910, 9f38cf89-56db-4c12-9fde-cbbbcd4fbbf9, 4de7a10c-eaab-4883-9354-6f9e7c10df76, c72443b6-e851-416d-9b87-070e79d2db61, d58d421d-43f7-442d-a069-c04e288f269a, 1645b718-385a-41c7-8274-46b5a8e1f91a, 4aa17c73-39a5-45c9-8e33-7d2b290bdc1e, 0cc055b9-8b9e-4920-8434-4759ac3e0c9a, 7a0f7154-df27-4cc4-aebb-dbb254f1129e